In [1]:
import sys
from pathlib import Path
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import Model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from dynamic_tcn import DynamicTCN
from classifier import dynamic_tcn_classifier

# data preprocessing functions


In [2]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
 
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix='../data/'):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix='../data/'):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'uci_har_dataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'uci_har_dataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy
 
    
def lstm_model(n_timesteps, n_features, n_outputs):
    model = Sequential()
    model.add(L.LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(L.Dropout(0.5))
    model.add(L.Dense(100, activation='relu'))
    model.add(L.Dense(n_outputs, activation='softmax'))
    return model

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, model, lr=1e-4):
    verbose, epochs, batch_size = 0, 15, 64
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy']
    )
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy
 
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 

def run_experiment(model_fn=lstm_model, lr=1e-3, repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        model = model_fn(trainX.shape[1], trainX.shape[2], trainy.shape[1])
        score = evaluate_model(trainX, trainy, testX, testy, model, lr)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
 


In [42]:
run_experiment(repeats=10)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 88.056
>#2: 90.261
>#3: 87.309
>#4: 72.989
>#5: 91.720
>#6: 90.601
>#7: 91.517
>#8: 90.261
>#9: 91.347
>#10: 91.347
[88.05565237998962, 90.26128053665161, 87.30912804603577, 72.98948168754578, 91.7203962802887, 90.60060977935791, 91.51679873466492, 90.26128053665161, 91.34713411331177, 91.34713411331177]
Accuracy: 88.541% (+/-5.371)


# TCN experiments

In [3]:
dtcn = DynamicTCN(use_dynamic_conv=False, use_cbam=True)

In [4]:
X = np.random.rand(100, 200, 3)
y = (np.random.rand(100) > 0.5).astype(np.uint8)

In [5]:
clf = dynamic_tcn_classifier(input_shape=(None, 3), batch_size=4)

In [6]:
clf.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="mse")

In [7]:
clf.fit(X, y, batch_size=4, epochs=1)

25/25 [==============================] - 42s 67ms/step - loss: 0.4772


In [7]:
clf.layers

# On dataset

In [10]:
from functools import partial

In [11]:
tcn_model = partial(DynamicTCN, use_dynamic_conv=False, use_cbam=True)
run_experiment(model_fn=tcn_model, repeats=10)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


AttributeError: 'DynamicTCN' object has no attribute 'compile'